In [ ]:
import SimpleITK as sitk
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
import os
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as k
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Conv2DTranspose,merge
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from keras import Input
import dicom
import pylab

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = k.flatten(y_true)
    y_pred_f = k.flatten(y_pred)
    intersection = k.sum(y_true_f * y_pred_f)
    return (2. * intersection ) / (k.sum(y_true_f) + k.sum(y_pred_f) )


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
def imcut(X):
    a = np.where(X>=0.5)
    n = a[0].shape[0]
    index1 = a[0][0]
    index2 = a[0][n-1]
    return index1,index2

In [ ]:
def improcess(X,y,patchsize,patchnum):
    s = patchsize
    index1,index2 = imcut(X[0])
    len = X.shape[0]*patchnum
    X_train = np.zeros([len,s,s])
    y_train = np.zeros([len,1])
    ncol = X[0].shape[1]
    for i in range(0,X.shape[0]):
        X_slice = X[i]
        #choosing 
        indexrow = np.random.randint( int(index1+s/2)+1 , int(index2-s/2)-1, size = patchnum ) 
        indexcol = np.random.randint( int(s/2)+1 , int(ncol-s/2)-1 , size = patchnum )
        for j in range(0,patchnum):
            X_train[j+i*patchnum,:,:]=X_slice[int(indexrow[j]-s/2):int(indexrow[j]+s/2),int(indexcol[j]-s/2):int(indexcol[j]+s/2)]
            y_train[j+i*patchnum,:] = y[i,indexrow[j],indexcol[j]]
    
    return index1,index2,X_train,y_train

In [ ]:
def test_improcess(X,y,patchsize,patchnum):
    s = patchsize
    index1 = 200
    index2 = 450
    index3 = 150
    index4 = 400
    len = X.shape[0]*patchnum
    X_train = np.zeros([len,s,s])
    y_train = np.zeros([len,1])
    ncol = X[0].shape[1]
    for i in range(0,X.shape[0]):
        X_slice = X[i]
        #choosing 
        indexrow = np.random.randint( int(index1+s/2)+1 , int(index2-s/2)-1, size = patchnum ) 
        indexcol = np.random.randint( int(index3+s/2)+1 , int(index4-s/2)-1 , size = patchnum )
        for j in range(0,patchnum):
            X_train[j+i*patchnum,:,:]=X_slice[int(indexrow[j]-s/2):int(indexrow[j]+s/2),int(indexcol[j]-s/2):int(indexcol[j]+s/2)]
            y_train[j+i*patchnum,:] = y[i,indexrow[j],indexcol[j]]
    
    return index1,index2,X_train,y_train

In [ ]:
def get_subdirectory(path):
    dirList = []
    files = os.listdir(path)
    for f in files:
        if(os.path.isdir(path + '/' + f)):
            if(f[0] == '.'):
                pass #排除隐藏文件夹
            else:
                dirList.append(path+ '/' + f)
    return dirList

def get_ap(path):
    img_pathList = []
    name = 'arterial phase'
    subpath = get_subdirectory(path)
    pathnumber = len(subpath)
    for i in range(pathnumber):
        img_pathList.append(subpath[i] + '/' + name)
    return img_pathList

In [ ]:
def labelcluster(X,y):
    middle = X[:,230:430,189:318]
    kmeans = KMeans(n_clusters = 4).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    index1 = np.where(X < centers[0])
    index2 = np.where((X >= centers[0])&(X < centers[1]))
    index3 = np.where((X >= centers[1])&(X < centers[2]))
    index4 = np.where((X >= centers[2])&(X < centers[3]))
    index5 = np.where(X >= centers[3])
    index0 = np.where(y == 1)#1代表白色
    y[index1] = 1
    y[index2] = 2
    y[index3] = 3
    y[index4] = 4
    y[index5] = 5
    y[index0] = 0
    return y

In [ ]:
def get_image(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

#读取当前文件夹下的所有DICOM文件，并以数组形式存储在image_array中，
def Xyread(path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_array = sitk.GetArrayFromImage(image)
    X = image_array
    mean = np.mean(X)
    std = np.std(X)
    X = X-mean
    X = X/std
    Xshape = X.shape
    
    c = get_image(path)
    d = len(c)
    
    mask = np.empty((d,512,512))
    i = 0
    while(d>0):
        img = Image.open(c[d-1])
        img_ndarray = np.asarray(img)/255
        mask[i] = img_ndarray
        d = d-1
        i = i+1
    return X,mask

In [ ]:
def path_getandcount(path,number_total = 0):
    path_total = get_ap(path)
    path_number = len(path_total)
    return path_total

In [ ]:
path = r"E:/Bdata/dataset"
path_total= get_ap(path)


In [ ]:
X1,mask1 = Xyread(path_total[0])
mask1 = labelcluster(X1,mask1)
#mask18 = mask1[8]
#X18 = X1[8]
#middle = X[:,230:430,189:318]
#kmeans = KMeans(n_clusters = 4).fit(np.reshape(middle,[np.prod(middle.shape),1]))
#centers = sorted(kmeans.cluster_centers_.flatten())
#index1 = np.where(X < centers[0])
#index2 = np.where((X > centers[0])&(X < centers[1]))
#index3 = np.where((X > centers[1])&(X < centers[2]))
#index4 = np.where(X > centers[3])
#index0 = np.where(y == 1)#1代表白色
#y[index1] = 1
#y[index2] = 2
#y[index3] = 3
#y[index4] = 4
#y[index0] = 0

In [ ]:
np.where(mask_show2 == 1)

In [ ]:
X,mask = Xyread(path_total[0])
#mask = labelcluster(X,mask)
for i in range(1,3):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X = np.append(X,X_temp,axis = 0)
    mask = np.append(mask,mask_temp,axis = 0)


In [ ]:
np.save("C:/Users/cassi/Desktop/mask_3.npy",mask)

In [ ]:
X1,mask1 = Xyread(path_total[20])
#mask = labelcluster(X,mask)
for i in range(21,40):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X1 = np.append(X1,X_temp,axis = 0)
    mask1 = np.append(mask1,mask_temp,axis = 0)

In [ ]:
X2,mask2 = Xyread(path_total[40])
#mask = labelcluster(X,mask)
for i in range(41,60):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X2 = np.append(X2,X_temp,axis = 0)
    mask2 = np.append(mask2,mask_temp,axis = 0)

In [ ]:
X3,mask3 = Xyread(path_total[60])
#mask = labelcluster(X,mask)
for i in range(61,80):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X3 = np.append(X3,X_temp,axis = 0)
    mask3 = np.append(mask3,mask_temp,axis = 0)

In [ ]:
X4,mask4 = Xyread(path_total[80])
#mask = labelcluster(X,mask)
for i in range(81,100):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X1 = np.append(X1,X_temp,axis = 0)
    mask1 = np.append(mask1,mask_temp,axis = 0)

In [ ]:
X5,mask5 = Xyread(path_total[100])
#mask = labelcluster(X,mask)
for i in range(101,107):
    X_temp,mask_temp = Xyread(path_total[i])
#    mask_temp = labelcluster(X_temp,mask_temp)
    X5 = np.append(X5,X_temp,axis = 0)
    mask5 = np.append(mask5,mask_temp,axis = 0)

In [ ]:
index=np.unique(np.where(mask==1)[0])
X_bw = X[index,:,:]
mask_bw = mask[index,:,:]

In [ ]:
index1=np.unique(np.where(mask1==1)[0])
X_bw1 = X1[index1,:,:]
mask1_bw = mask1[index1,:,:]
index2=np.unique(np.where(mask2==1)[0])
X_bw2 = X2[index2,:,:]
mask2_bw = mask2[index2,:,:]
index3=np.unique(np.where(mask3==1)[0])
X_bw3 = X3[index3,:,:]
mask3_bw = mask3[index3,:,:]
index4=np.unique(np.where(mask4==1)[0])
X_bw4 = X4[index4,:,:]
mask4_bw = mask4[index4,:,:]

In [ ]:
X_input_bw = np.concatenate((X_bw,X_bw1,X_bw2,X_bw3,X_bw4),axis = 0)

In [ ]:
np.save("C:/Users/cassi/Desktop/X.npy",X)
np.save("C:/Users/cassi/Desktop/X1.npy",X1)
np.save("C:/Users/cassi/Desktop/X2.npy",X2)
np.save("C:/Users/cassi/Desktop/X3.npy",X3)

In [ ]:
np.save("C:/Users/cassi/Desktop/X4.npy",X4)
np.save("C:/Users/cassi/Desktop/X5.npy",X5)
np.save("C:/Users/cassi/Desktop/mask.npy",mask)
np.save("C:/Users/cassi/Desktop/mask1.npy",mask1)
np.save("C:/Users/cassi/Desktop/mask2.npy",mask2)
np.save("C:/Users/cassi/Desktop/mask3.npy",mask3)
np.save("C:/Users/cassi/Desktop/mask4.npy",mask4)
np.save("C:/Users/cassi/Desktop/mask5.npy",mask5)

In [ ]:
mask = np.load('C:/Users/cassi/Desktop/mask.npy')

In [ ]:
mask_input = np.concatenate((mask,mask1,mask2,mask3,mask4),axis = 0)

In [ ]:
np.save('C:/Users/cassi/Desktop/Image_input_all.npy',X_input)

In [ ]:
index1,index2,X_output,y_output = improcess(X,mask,21,500)
num_category = 2
y_output = keras.utils.to_categorical(y_output, num_category)
#index_mask =  np.where(y_output[:,0] == 1)
#index_mask
#index_tumor = index_mask[0]
#X_output1 = X_output[index_tumor]
#y_output1 = y_output[index_tumor]

image_number = X_output.shape[0]
split = int(0.8*image_number)

y_test = y_output[split:image_number,:]
y_train = y_output[0:split,:]


In [ ]:
X_output.shape

In [ ]:
X_test = X_output[split:image_number,:,:]
X_train = X_output[0:split,:,:]
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
if k.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
#more reshaping
#X_train.shape
#X_train = X_train.astype('float64')
#X_test = X_test.astype('float64')
#X_train = X_train - X_train.min()
#X_train = X_train/X_train.max()
#X_test = X_test - X_test.min()
#X_test = X_test/X_test.max()
print('X_train shape:', X_train.shape,X_test.shape,y_train.shape,y_test.shape) #X_train shape: (60000, 28, 28, 1)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_category, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=[dice_coef])

In [ ]:
batch_size = 128
num_epoch = 6
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
model.save("C:/Users/cassi/Desktop/CNNmodel_15_3_21_500.h5")

In [ ]:
from keras.models import load_model
model1 = load_model("C:/Users/cassi/Desktop/CNNmodel_6_3_21_500.h5")

In [ ]:
path1 = "C:/Users/cassi/Desktop/10011.dcm"
image_predict = sitk.ReadImage(path1)
X_predict_array = sitk.GetArrayFromImage(image_predict)[0]
patchsize = 21

mean = np.mean(X_predict_array)
std = np.std(X_predict_array)
X_predict_array = X_predict_array-mean
X_predict_array = X_predict_array/std
middle = X_predict_array[230:430,189:318]
kmeans = KMeans(n_clusters = 4).fit(np.reshape(middle,[np.prod(middle.shape),1]))
centers = sorted(kmeans.cluster_centers_.flatten())
index1 = np.where(X_predict_array < centers[0])
index2 = np.where((X_predict_array > centers[0])&(X_predict_array < centers[1]))
index3 = np.where((X_predict_array > centers[1])&(X_predict_array < centers[2]))
index4 = np.where(X_predict_array > centers[3])

#X_predict_index = imcut(X_predict_array)
#w = int(patchsize/2)
#X_predict = np.zeros([(X_predict_index[1]-X_predict_index[0])*(512-patchsize+1),patchsize,patchsize,1])
#index = 0
#for i in range(X_predict_index[0],min(X_predict_index[1],512-w)):
#    for j in range(w,512-w):
#        X_predict[index,:,:,0] = X_predict_array[i-w:i+w+1,j-w:j+w+1]
#        index = index+1


#i = 349
#modelpredict = np.zeros([512,512,2])
#for i in range(337,380):
#    for j in range(218,254):
#        modelpredict[i,j,:] = model.predict(X_predict_array[0,i-w:i+w+1,j-w:j+w+1].reshape(1,13,13,1))

#i,j=10,10
#model.predict(X_predict_array[0,i-w:i+w+1,j-w:j+w+1].reshape(1,13,13,1))

In [ ]:
centers

In [ ]:
X_predict_show = np.zeros([512,512])
X_predict_show[index4] = 1
plt.imshow(X_predict_show,'gray')
plt.show()

In [ ]:
X_predict_index = [230,430,189,318]
w = int(patchsize/2)
X_predict = np.zeros([(X_predict_index[1]-X_predict_index[0])*(X_predict_index[3]-X_predict_index[2]),patchsize,patchsize,1])
index = 0
for i in range(X_predict_index[0],min(X_predict_index[1],512-w)):
    for j in range(X_predict_index[2],X_predict_index[3]):
        X_predict[index,:,:,0] = X_predict_array[i-w:i+w+1,j-w:j+w+1]
        index = index+1

In [ ]:
y_predict = model.predict(X_predict)

In [ ]:
y_predict.shape

In [ ]:
y_pi = y_predict[:,1].reshape(X_predict_index[1]-X_predict_index[0],512-patchsize+1)#reshape按行重排
#y_pi2 = y_predict[:,2].reshape(X_predict_index[1]-X_predict_index[0],512-patchsize+1)
#y_pi = y_predict[:,0].reshape(X_predict_index[1]-X_predict_index[0],X_predict_index[3]-X_predict_index[2])

y_po = np.zeros([X_predict_index[1]-X_predict_index[0],512-patchsize+1])
y_show = np.zeros([512,512])
output_index = np.where(y_pi == y_pi.max() )
#output_index2 = np.where(y_pi2 > 0 )
y_po[output_index] = 1
#y_po[output_index2] = 2
y_show[X_predict_index[0]:X_predict_index[1],int(patchsize/2):512-int(patchsize/2)] = y_po

